# 모델 설계도

#### 1. 2017 ~ 2019년의 3년간의 자료가 있는 종목은 2549 종목

#### 2. 그 중에 3년치 데이터(733 obs)가 온전하게 있는 경우는 2324 종목

#### 3. 5일 후를 예측하는 모형 생성 준비

#### 4. 그 종목들 중 10%에 대하여 Test로 분리

#### 5. 90%의 Train 데이터 중 다시 90 : 10 으로 Train과 Test를 랜덤으로 분리

#### 6. Random Forest 모형을 적용

#### 7. 10%의 Test 종목에 대하여 평균 RMSE 계산

In [1]:
# Seed Fix
from numpy.random import seed
seed(1234)
from tensorflow import set_random_seed
set_random_seed(1234)

In [2]:
# Warning Message
import warnings
warnings.filterwarnings(action='ignore') 

# Data Loading
import pandas as pd
data = pd.read_csv("./data/df_final.csv")

# 필요한 칼럼 추출
data = data[["date", "open", "high", "low", "close", "code"]]

# Code 앞에 0 추가
data["code"] = data["code"].apply(lambda x : "{:0>6}" .format(x))

# Code type을 str로 변경
data["code"] = data["code"].apply(lambda x : str(x))

# 각 종목별 obs 개수
print("*"*22)
print("* 각 종목별 obs 개수 *")
print("*"*22)
print(data["code"].value_counts())

print("\n\n")

# 3년간의 데이터가 완전한 종목 코드
print("*"*42)
print("* 3년간 데이터가 완전한 종목 코드 733obs *")
print("*"*42)
FULL_DATA_CODE = data["code"].value_counts().index[0:2323]
print(FULL_DATA_CODE)

print("\n\n")

# 3년간의 데이터가 완전한 종목만 추출한 데이터
print("*"*48)
print("* 3년간의 데이터가 완전한 종목만 추출한 데이터 *")
print("*"*48)
data = data[data["code"].isin(FULL_DATA_CODE)]
print(data)

print("\n\n")

# RESULT DATASET 생성
FINAL_DATA = pd.DataFrame()

# 각 종목 별 Shift 생성
# 5일 전 종가로 5일 후 종가를 예측
temp_data = data                                                                                      
for i in FULL_DATA_CODE[0:3] :                                                                      
    data = data[data["code"] == i]                                                                   
    for s in range(1,6):                                                                             
            data['{}{}'.format("close",s)] = data["close"].shift(s)                                        
    data = data.dropna().drop(["open", "high", "low", "close1", "close2", "close3", "close4", "date", "code"], axis=1)
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    scaler.fit(data)
    data = scaler.transform(data)
    data = pd.DataFrame(data, columns = ["close", "close5"])
    FINAL_DATA = FINAL_DATA.append(data)                                                             
    data = temp_data                                                                               
print("*"*49)
print("* SHIFT 생성 : 5일 전 종가로 5일 후 종가를 예측 *")
print("*"*49)
FINAL_DATA = FINAL_DATA[["close", "close5"]]
print("DATA Length : {}obs" .format(len(FINAL_DATA)))
print(FINAL_DATA[0:10])

**********************
* 각 종목별 obs 개수 *
**********************
590016    733
530021    733
064260    733
014100    733
042520    733
         ... 
287310    494
570024    494
287300    494
287320    494
269620    493
Name: code, Length: 2539, dtype: int64



******************************************
* 3년간 데이터가 완전한 종목 코드 733obs *
******************************************
Index(['590016', '530021', '064260', '014100', '042520', '049120', '008290',
       '009270', '069660', '039840',
       ...
       '001799', '155900', '006920', '002787', '117580', '011280', '214870',
       '080530', '150460', '550009'],
      dtype='object', length=2323)



************************************************
* 3년간의 데이터가 완전한 종목만 추출한 데이터 *
************************************************
               date   open   high    low  close    code
0        2017-01-02  79905  81585  79170  79800  012320
1        2017-01-03  80535  80535  79170  79800  012320
2        2017-01-04  80535  80955  76965  77175  01

In [3]:
# Data 새로 만들기
FINAL_DATA.to_csv("./data/ALLCODE_DATA.csv")

# 새로 만든 데이터 불러오기
data = pd.read_csv("./data/ALLCODE_DATA.csv")

data = data[['close', 'close5']]

In [4]:
# Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data[["close5"]], data[["close"]], test_size=0.10, random_state=42)

# Model Fitting
from sklearn.linear_model import LinearRegression
reg = LinearRegression(n_jobs = -1).fit(X_train, y_train)

print("="*40)
print("{:=^40}" .format(" Linear Regression "))
print("="*40)
print("Train Score : {}" .format(reg.score(X_train, y_train)))
print("Test Score : {}" .format(reg.score(X_test, y_test)))

# RMSE
import numpy as np
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(y_test, reg.predict(X_test))
print("RMSE : {}" .format(rmse))
print("Percent RMSE : {}" .format(rmse / np.mean(y_test)))
print("="*40)

========== Linear Regression ===========
Train Score : 0.9110373816505866
Test Score : 0.915809897172243
RMSE : 0.0032008393433941625
Percent RMSE : close    0.006591
dtype: float64
